In [1]:
from pyspark import SparkContext
sc = SparkContext()
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
spark

In [4]:
df = spark.read.csv('data/sales.csv', header=True, inferSchema=True)

In [7]:
df.show(5)

+----------------+--------+-----+------------+-----------------+--------------------+--------+--------------+----------------+----------------+-----------+----------+
|Transaction_date| Product|Price|Payment_Type|             Name|                City|   State|       Country| Account_Created|      Last_Login|   Latitude| Longitude|
+----------------+--------+-----+------------+-----------------+--------------------+--------+--------------+----------------+----------------+-----------+----------+
| 01/02/2009 6:17|Product1| 1200|  Mastercard|         carolina|            Basildon| England|United Kingdom| 01/02/2009 6:00| 01/02/2009 6:08|       51.5|-1.1166667|
| 01/02/2009 4:53|Product1| 1200|        Visa|           Betina|Parkville        ...|      MO| United States| 01/02/2009 4:42| 01/02/2009 7:49|     39.195| -94.68194|
|01/02/2009 13:08|Product1| 1200|  Mastercard|Federica e Andrea|Astoria          ...|      OR| United States|01/01/2009 16:21|01/03/2009 12:32|   46.18806|   -123.83

In [8]:
df.select('Country').distinct().show()

+-------------+
|      Country|
+-------------+
|       Sweden|
|       Jersey|
|     Malaysia|
|       Turkey|
|      Germany|
|       France|
|      Belgium|
|      Finland|
|United States|
|        India|
|       Kuwait|
|        Malta|
|        Italy|
|       Norway|
|        Spain|
|      Denmark|
|      Ireland|
|       Israel|
|      Iceland|
|  South Korea|
+-------------+
only showing top 20 rows



In [12]:
df.filter("Country = 'Brazil'").select('Name', 'Price').show()

+-------+-----+
|   Name|Price|
+-------+-----+
|Joachim| 1200|
|  Diana| 7500|
+-------+-----+



In [14]:
df.groupBy('Country').sum('Price') \
  .withColumnRenamed('sum(Price)', 'TotalPrice') \
  .show()

+-------------+----------+
|      Country|TotalPrice|
+-------------+----------+
|       Sweden|      8400|
|       Jersey|      1200|
|     Malaysia|      1200|
|       Turkey|      2400|
|      Germany|     22800|
|       France|     30300|
|      Belgium|      3600|
|      Finland|      1200|
|United States|    350350|
|        India|      2400|
|       Kuwait|      1200|
|        Malta|      3600|
|        Italy|      2400|
|       Norway|     12000|
|        Spain|      2400|
|      Denmark|      8400|
|      Ireland|     29100|
|       Israel|      1200|
|      Iceland|      1200|
|  South Korea|      1200|
+-------------+----------+
only showing top 20 rows



In [23]:
df.groupBy('Country').sum('Price') \
  .withColumnRenamed('sum(Price)', 'TotalPrice') \
  .orderBy('TotalPrice', ascending = False) \
  .show()

+--------------------+----------+
|             Country|TotalPrice|
+--------------------+----------+
|       United States|    350350|
|      United Kingdom|     63600|
|              Canada|     42000|
|              France|     30300|
|             Ireland|     29100|
|             Germany|     22800|
|           Australia|     22800|
|         Switzerland|     19200|
|         Netherlands|     14400|
|              Norway|     12000|
|              Brazil|      8700|
|              Sweden|      8400|
|             Denmark|      8400|
|             Austria|      3600|
|        South Africa|      3600|
|United Arab Emirates|      3600|
|             Belgium|      3600|
|               Malta|      3600|
|               India|      2400|
|               Italy|      2400|
+--------------------+----------+
only showing top 20 rows



In [24]:
df2 = spark.read.csv('data/countries.csv', header=True, inferSchema=True)

In [25]:
df2.show(5)

+--------------+---+
|       Country| ID|
+--------------+---+
|United Kingdom|  1|
| United States|  2|
|     Australia|  3|
|        Israel|  4|
|        France|  5|
+--------------+---+
only showing top 5 rows



In [31]:
df2.join(df, 'Country').groupBy('Country').sum('Price').show()

+-------------+----------+
|      Country|sum(Price)|
+-------------+----------+
|       Sweden|      8400|
|       Jersey|      1200|
|     Malaysia|      1200|
|       Turkey|      2400|
|      Germany|     22800|
|       France|     30300|
|      Belgium|      3600|
|      Finland|      1200|
|United States|    350350|
|        India|      2400|
|       Kuwait|      1200|
|        Malta|      3600|
|        Italy|      2400|
|       Norway|     12000|
|        Spain|      2400|
|      Denmark|      8400|
|      Ireland|     29100|
|       Israel|      1200|
|      Iceland|      1200|
|  South Korea|      1200|
+-------------+----------+
only showing top 20 rows



In [33]:
df.groupBy('Country').sum('Price') \
  .join(df2, 'Country') \
  .withColumnRenamed('sum(Price)', 'TotalPrice') \
  .select('ID', 'TotalPrice') \
  .show()

"""
df.join(df2, 'Country') \
  .groupBy('ID').sum('Price') \
  .withColumnRenamed('sum(ID)', 'TotalPrice') \
  .show()
"""

+---+----------+
| ID|TotalPrice|
+---+----------+
| 15|      8400|
| 29|      1200|
| 30|      1200|
| 24|      2400|
| 19|     22800|
|  5|     30300|
| 14|      3600|
| 11|      1200|
|  2|    350350|
|  9|      2400|
| 25|      1200|
| 26|      3600|
| 18|      2400|
| 16|     12000|
| 21|      2400|
| 13|      8400|
|  7|     29100|
|  4|      1200|
| 31|      1200|
| 32|      1200|
+---+----------+
only showing top 20 rows



In [10]:
from pyspark.sql.functions import *

numOfIterations = 10

lines = spark.read.text("data/pagerank_data.txt")
# You can also test your program on the follow larger data set:
# lines = spark.read.text("dblp.in")

a = lines.select(split(lines[0],' '))

links = a.select(a[0][0].alias('src'), a[0][1].alias('dst'))
links.show()
outdegrees = links.groupBy('src').count()

outdegrees = outdegrees.select('src', 'count')

ranks = outdegrees.select('src', lit(1).alias('rank')) # lit(1)是初始化为1的意思

# number of objects --> num = 4
num = ranks.count()

"""
for iteration in range(numOfIterations):
    ranks = links.join(outdegrees, 'src').join(ranks, 'src') \
             .select('*', (ranks['rank'] / outdegrees['count'] * 0.85 + 0.15 / num).alias('next_rank')) \
             .withColumnRenamed('dst','dst1').groupBy('dst1').sum('next_rank') \
             .withColumnRenamed('sum(next_rank)', 'rank').withColumnRenamed('dst1', 'src')
"""

for iteration in range(numOfIterations):
    contribs = links.join(outdegrees, 'src').join(ranks, 'src') \
                    .select('*', (ranks['rank'] / outdegrees['count']).alias('contribs')) \
                    .withColumnRenamed('dst','dst1').groupBy('dst1').sum('contribs')
    
    ranks = contribs.select ('dst1', (contribs['sum(contribs)'] * 0.85 + 0.15 / num).alias('rank')) \
                    .withColumnRenamed('dst1', 'src')
"""
for iteration in range(numOfIterations):
    ranks = links.join(outdegrees, 'src').join(ranks, 'src') \
             .select('*', (ranks['rank'] / outdegrees['count'] * 0.85 + 0.15 / num).alias('next_rank')) \
             .groupBy('dst').sum('next_rank') \
             .withColumnRenamed('sum(next_rank)', 'rank').withColumnRenamed('dst', 'src')
"""
    
# 第三行有bug

ranks.orderBy(desc('rank')).show()

+---+---+
|src|dst|
+---+---+
|  1|  2|
|  1|  3|
|  2|  3|
|  3|  4|
|  4|  1|
|  2|  1|
+---+---+

+---+-------------------+
|src|               rank|
+---+-------------------+
|  1| 0.5213734076219673|
|  3| 0.3976558032555418|
|  4|0.39765580325554173|
|  2| 0.2739381988891164|
+---+-------------------+

